In [1]:
from keras.applications import EfficientNetB0
from keras.applications.efficientnet import preprocess_input
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
from keras.datasets import cifar10
import numpy as np
import cv2

# Load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Resize images to 224x224 and preprocess
x_train = np.array([cv2.resize(img, (224, 224)) for img in x_train])
x_test = np.array([cv2.resize(img, (224, 224)) for img in x_test])

# Normalize using EfficientNet's preprocessing
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create validation split
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1, random_state=42
)

In [3]:
base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_tensor=Input(shape=(224, 224, 3))
)

base_model.trainable = False  # Freeze base layers initially

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=5,
    batch_size=32,
    verbose=2
)

Epoch 1/5


/opt/anaconda3/envs/cnn/lib/python3.9/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(


1407/1407 - 657s - 467ms/step - accuracy: 0.8677 - loss: 0.3947 - val_accuracy: 0.9024 - val_loss: 0.2780
Epoch 2/5
1407/1407 - 681s - 484ms/step - accuracy: 0.9019 - loss: 0.2826 - val_accuracy: 0.9094 - val_loss: 0.2600
Epoch 3/5
1407/1407 - 696s - 495ms/step - accuracy: 0.9160 - loss: 0.2418 - val_accuracy: 0.9112 - val_loss: 0.2444
Epoch 4/5
1407/1407 - 4198s - 3s/step - accuracy: 0.9251 - loss: 0.2118 - val_accuracy: 0.9146 - val_loss: 0.2337
Epoch 5/5
1407/1407 - 5676s - 4s/step - accuracy: 0.9332 - loss: 0.1884 - val_accuracy: 0.9162 - val_loss: 0.2470


In [6]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

Test accuracy: 0.9073


In [7]:
base_model.trainable = True  # Unfreeze all layers (or select a range)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Lower LR
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          epochs=3,
          batch_size=32,
          verbose=2)

Epoch 1/3
1407/1407 - 11480s - 8s/step - accuracy: 0.7994 - loss: 0.6166 - val_accuracy: 0.8850 - val_loss: 0.3359
Epoch 2/3
1407/1407 - 23754s - 17s/step - accuracy: 0.8856 - loss: 0.3391 - val_accuracy: 0.9130 - val_loss: 0.2536
Epoch 3/3
1407/1407 - 17467s - 12s/step - accuracy: 0.9101 - loss: 0.2653 - val_accuracy: 0.9278 - val_loss: 0.2085


In [9]:
11480/60


191.33333333333334